<a href="https://colab.research.google.com/github/kaushanr/ml-notes-n-projects/blob/main/CH_02_Q.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
# Downloading the dataset

import os
import tarfile
import urllib

download_root = 'https://raw.githubusercontent.com/ageron/handson-ml2/master/'
housing_path = os.path.join('datasets','housing')
housing_url = download_root + 'datasets/housing/housing.tgz'

def fetch_housing_data(url=housing_url, path=housing_path):
  os.makedirs(path,exist_ok=True) # raises FileExistsError if directories and path already exists
  tgz_path = os.path.join(path,'housing.tgz')
  urllib.request.urlretrieve(url,tgz_path)
  housing_tgz = tarfile.open(tgz_path)
  housing_tgz.extractall(path)
  housing_tgz.close()

fetch_housing_data()

# Load the fetched data into a pandas.DataFrame

import pandas as pd

def load_housing_data(path=housing_path):
  csv_path = os.path.join(path,'housing.csv')
  return pd.read_csv(csv_path)

housing = load_housing_data()

print(housing.info()) # checking for null values in the data records
housing.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB
None


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [93]:
# Segmenting train-test subsets from dataset

import numpy as np

# Option with Stratified Sampling 

from sklearn.model_selection import StratifiedShuffleSplit as strat_samp

housing['income_cat'] = pd.cut(housing['median_income'],
                               bins=[0.,1.5,3.,4.5,6.,np.inf],
                               labels=[1,2,3,4,5])

split_data = strat_samp(n_splits=1,test_size=0.2,random_state=42)

for train_index, test_index in split_data.split(housing,housing['income_cat']):
  strat_train_set = housing.loc[train_index] # .loc - location indexer - Access a group of rows and columns by label(s) or a boolean array.
  strat_test_set = housing.loc[test_index]

for set_ in (strat_train_set,strat_test_set):
  set_.drop('income_cat',axis=1,inplace=True)

housing = strat_train_set.drop('median_house_value',axis=1)
housing_labels = strat_train_set['median_house_value'].copy()

strat_train_set.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
12655,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,72100.0,INLAND
15502,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,279600.0,NEAR OCEAN
2908,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750,82700.0,INLAND
14053,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,112500.0,NEAR OCEAN
20496,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,238300.0,<1H OCEAN


In [94]:
# Custom Transformer 1

from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, households_ix = 3,4,5,6

class CombinedAttributesAdder(BaseEstimator,TransformerMixin):
  
  def __init__(self,add_bedrooms_per_room=True):
    self.add_bedrooms_per_room = add_bedrooms_per_room

  def fit(self, X,y=None):
    return self

  def transform(self,X):
    rooms_per_household = X[:,population_ix] / X[:,households_ix] # X[:,4] where : - all rows, 4 - of the fourth column 
    population_per_household = X[:,population_ix] / X[:,households_ix]
    if self.add_bedrooms_per_room:
      bedrooms_per_room = X[:,bedrooms_ix] / X[:,rooms_ix]
      return np.c_[X,rooms_per_household,population_per_household,bedrooms_per_room] # adds all the new feature columns back along with the original attributes
    else:
      return np.c_[X,rooms_per_household,population_per_household]

#attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
#housing_extra_attribs= attr_adder.transform(housing.values)


In [95]:
# Pipeline

# Transformation Pipelines

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

housing_num = housing.drop('ocean_proximity',axis=1)

num_pipeline = Pipeline([
    ('imputer',SimpleImputer(strategy='median')),
    ('attribs_adder',CombinedAttributesAdder()),
    ('std_scaler',StandardScaler())
])

#housing_num_tr = num_pipeline.fit_transform(housing_num)

from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ['ocean_proximity']

prep_pipeline = ColumnTransformer([
    ('num',num_pipeline,num_attribs),
    ('cat',OneHotEncoder(handle_unknown = 'ignore'),cat_attribs)
])

housing_prepared = prep_pipeline.fit_transform(housing) # <------ first data.fit method required for feeding results to feature selector

In [96]:
# Best Feature Selector Model Transformer

# ML Features Selector

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

prep_pipeline.named_transformers_['cat'].handle_unknown = 'ignore'
print(prep_pipeline.named_transformers_['cat'])

param_grid = [
    {'n_estimators':[3,10,30],'max_features':[2,4,6,8]},
    {'bootstrap':[False],'n_estimators':[3,10],'max_features':[2,3,4]},
]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg,param_grid,cv=5,scoring='neg_mean_squared_error',return_train_score=True,verbose=2)
grid_search.fit(housing_prepared,housing_labels)


print(grid_search.best_params_)
print(grid_search.best_estimator_)

# Analyze the best hyperparameters/features and their errors

feature_importances = grid_search.best_estimator_.feature_importances_
print(feature_importances,'\n')

extra_attribs = ['rooms_per_hhold','pop_per_hhold','bedrooms_per_room']
cat_encoder = prep_pipeline.named_transformers_['cat']
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
print(sorted(zip(feature_importances,attributes),reverse=True))

# Best Feature Selector Transformer

from sklearn.base import BaseEstimator, TransformerMixin

def indices_of_top_k(arr, k):
    return np.sort(np.argpartition(np.array(arr), -k)[-k:]) # Element index to partition by. The k-th element will be in its
                                                            # final sorted position and all smaller elements will be moved
                                                            # before it and all larger elements behind it. The order all
                                                            # elements in the partitions is undefined. If provided with a
                                                            # sequence of k-th it will partition all of them into their sorted
                                                            # position at once.

                                                            #             ascending(LSB)<--------|--->ascending(MSB)                                  
                                                            # [11 15  2  3  4  5  6 14  8  9 13  0 12 10  1  7]

# Custom Transformer 2

class TopFeatureSelector(BaseEstimator, TransformerMixin):

    def __init__(self, feature_importances, k):
        self.feature_importances = feature_importances
        self.k = k

    def fit(self, X, y=None):
        self.feature_indices_ = indices_of_top_k(self.feature_importances, self.k)
        return self

    def transform(self, X):
        return X[:, self.feature_indices_]

k = 5 # SELECT most significant parameters

select_best_features = Pipeline([
    ('preparation', prep_pipeline),
    ('feature_selection', TopFeatureSelector(feature_importances, k))
])


top_k_feature_indices = indices_of_top_k(feature_importances, k)
top_k_feature_indices

np.array(attributes)[top_k_feature_indices]
sorted(zip(feature_importances, attributes), reverse=True)[:k]

housing_prepared_top_features = select_best_features.fit_transform(housing)

OneHotEncoder(handle_unknown='ignore')
Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END .....................max_features=2, n_estimators=3; total time=   0.1s
[CV] END .....................max_features=2, n_estimators=3; total time=   0.1s
[CV] END .....................max_features=2, n_estimators=3; total time=   0.1s
[CV] END .....................max_features=2, n_estimators=3; total time=   0.1s
[CV] END .....................max_features=2, n_estimators=3; total time=   0.1s
[CV] END ....................max_features=2, n_estimators=10; total time=   0.2s
[CV] END ....................max_features=2, n_estimators=10; total time=   0.2s
[CV] END ....................max_features=2, n_estimators=10; total time=   0.2s
[CV] END ....................max_features=2, n_estimators=10; total time=   0.2s
[CV] END ....................max_features=2, n_estimators=10; total time=   0.2s
[CV] END ....................max_features=2, n_estimators=30; total time=   0.6s
[CV] END 

In [97]:
# Training ML Model

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, reciprocal
from sklearn.svm import SVR

# see https://docs.scipy.org/doc/scipy/reference/stats.html
# for `expon()` and `reciprocal()` documentation and more probability distribution functions.

# Note: gamma is ignored when kernel is "linear"
#param_distribs = {
#        'kernel': ['linear', 'rbf'],
#        'C': reciprocal(20, 200000),
#        'gamma': expon(scale=1.0),
#    }

#svm_reg = SVR()
#rnd_search = RandomizedSearchCV(svm_reg, param_distributions=param_distribs,
#                                n_iter=3, cv=2, scoring='neg_mean_squared_error',
#                                verbose=2, random_state=42)
#rnd_search.fit(housing_prepared_top_features, housing_labels)

################################################################################

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

prep_pipeline.named_transformers_['cat'].handle_unknown = 'ignore'
print(prep_pipeline.named_transformers_['cat'])

param_grid = [
    {'n_estimators':[100,300],'max_features':list(range(1, k+1))},
    {'bootstrap':[False],'n_estimators':[100,300],'max_features':list(range(1, k+1))},
]

forest_reg2 = RandomForestRegressor()
grid_search_model = GridSearchCV(forest_reg2,param_grid,cv=5,scoring='neg_mean_squared_error',return_train_score=True,verbose=2)
grid_search_model.fit(housing_prepared_top_features,housing_labels)


print(grid_search_model.best_params_)

################################################################################

full_pipeline = Pipeline([
    ('best_features', select_best_features),
    #('svm_reg', SVR(**rnd_search.best_params_)),
    ('forest_2_reg', RandomForestRegressor(**grid_search_model.best_params_))
])


full_pipeline.fit(housing,housing_labels)

OneHotEncoder(handle_unknown='ignore')
Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END ...................max_features=1, n_estimators=100; total time=   1.4s
[CV] END ...................max_features=1, n_estimators=100; total time=   1.4s
[CV] END ...................max_features=1, n_estimators=100; total time=   1.4s
[CV] END ...................max_features=1, n_estimators=100; total time=   1.4s
[CV] END ...................max_features=1, n_estimators=100; total time=   1.4s
[CV] END ...................max_features=1, n_estimators=300; total time=   4.1s
[CV] END ...................max_features=1, n_estimators=300; total time=   4.1s
[CV] END ...................max_features=1, n_estimators=300; total time=   4.1s
[CV] END ...................max_features=1, n_estimators=300; total time=   4.1s
[CV] END ...................max_features=1, n_estimators=300; total time=   4.1s
[CV] END ...................max_features=2, n_estimators=100; total time=   2.0s
[CV] END

Pipeline(steps=[('best_features',
                 Pipeline(steps=[('preparation',
                                  ColumnTransformer(transformers=[('num',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(strategy='median')),
                                                                                   ('attribs_adder',
                                                                                    CombinedAttributesAdder()),
                                                                                   ('std_scaler',
                                                                                    StandardScaler())]),
                                                                   ['longitude',
                                                                    'latitude',
                                                    

In [98]:
# Save pipeline to pickle

import pickle

def save_model(pipeline):
  with open('model_ch2_a.pickle','wb') as file: # saved in .pickle format - 'wb' means write in binary
    pickle.dump((pipeline),file) # pickling multiple objects passed in through a tuple


save_model(full_pipeline)

In [99]:
negative_mse = grid_search_model.best_score_
rmse = np.sqrt(-negative_mse)
print(rmse)

print(grid_search_model.best_params_)

some_data = housing.iloc[:4]
some_labels = housing_labels.iloc[:4]

results = full_pipeline.predict(some_data)

print("Predictions:\t", results)
print("Labels:\t\t", list(some_labels))



57504.31511029146
{'max_features': 1, 'n_estimators': 300}
Predictions:	 [ 75676.         291769.67        85537.66666667 139080.66666667]
Labels:		 [72100.0, 279600.0, 82700.0, 112500.0]


In [100]:
# Evaluate model

# NOTE! - call transform() - not fit_transform() - you do not want to fit the test set

from sklearn.metrics import mean_squared_error

final_model = full_pipeline

X_test = strat_test_set.drop('median_house_value',axis=1)
y_test = strat_test_set["median_house_value"].copy()

final_predictions = final_model.predict(X_test)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print('Final RMSE : ', final_rmse)

from scipy import stats

confidence = 0.95

squared_errors = (final_predictions - y_test)**2
confidence_result = np.sqrt(stats.t.interval(confidence,len(squared_errors) - 1,
                         loc = squared_errors.mean(),
                         scale = stats.sem(squared_errors)))

print(f'95% Confidence Interval : {confidence_result}')
print('Confidence : ',round((1-(final_rmse - confidence_result[0])/confidence_result[0])*100,1),'%')

Final RMSE :  55623.651234421486
95% Confidence Interval : [53520.46623593 57650.15912542]
Confidence :  96.1 %


In [101]:
prep_pipeline.named_transformers_['cat'].handle_unknown = 'ignore'
prep_pipeline.named_transformers_['cat']

OneHotEncoder(handle_unknown='ignore')

In [104]:
# Using GridSearchCV to insert and try different params to the feature_selection and imputer transformers

param_grid = [{
    'best_features__preparation__num__imputer__strategy': ['mean', 'median', 'most_frequent'],
    'best_features__feature_selection__k': list(range(1, len(feature_importances)))
}]

grid_search_prep = GridSearchCV(full_pipeline, param_grid, cv=5,scoring='neg_mean_squared_error', verbose=2)

grid_search_prep.fit(housing,housing_labels)

    ## NOTE - Important
      # There should be two underscores between estimator name and 
      # it's parameters in a Pipeline logisticregression__C. Do the same for tfidfvectorizer

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END best_features__feature_selection__k=1, best_features__preparation__num__imputer__strategy=mean; total time=   4.4s
[CV] END best_features__feature_selection__k=1, best_features__preparation__num__imputer__strategy=mean; total time=   4.4s
[CV] END best_features__feature_selection__k=1, best_features__preparation__num__imputer__strategy=mean; total time=   4.4s
[CV] END best_features__feature_selection__k=1, best_features__preparation__num__imputer__strategy=mean; total time=   4.4s
[CV] END best_features__feature_selection__k=1, best_features__preparation__num__imputer__strategy=mean; total time=   4.4s
[CV] END best_features__feature_selection__k=1, best_features__preparation__num__imputer__strategy=median; total time=   4.4s
[CV] END best_features__feature_selection__k=1, best_features__preparation__num__imputer__strategy=median; total time=   4.3s
[CV] END best_features__feature_selection__k=1, best_features__pre

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
6 fits failed out of a total of 225.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 355, in _fit
    **fit_params_steps[name],
  File "/usr/local/lib/python3.7/dist-packages/joblib/memory.py", line 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('best_features',
                                        Pipeline(steps=[('preparation',
                                                         ColumnTransformer(transformers=[('num',
                                                                                          Pipeline(steps=[('imputer',
                                                                                                           SimpleImputer(strategy='median')),
                                                                                                          ('attribs_adder',
                                                                                                           CombinedAttributesAdder()),
                                                                                                          ('std_scaler',
                                                                                                           Standard

In [105]:
print(grid_search_prep.best_params_)

{'best_features__feature_selection__k': 6, 'best_features__preparation__num__imputer__strategy': 'mean'}


In [110]:
# Grid Search Optimized Final Model

optimized_final_model = grid_search_prep

X_test = strat_test_set.drop('median_house_value',axis=1)
y_test = strat_test_set["median_house_value"].copy()

optimized_final_predictions = optimized_final_model.predict(X_test)

final_mse = mean_squared_error(y_test, optimized_final_predictions)
final_rmse = np.sqrt(final_mse)
print('Final RMSE : ', final_rmse)

from scipy import stats

confidence = 0.95

squared_errors = (optimized_final_predictions - y_test)**2
confidence_result = np.sqrt(stats.t.interval(confidence,len(squared_errors) - 1,
                         loc = squared_errors.mean(),
                         scale = stats.sem(squared_errors)))

print(f'95% Confidence Interval : {confidence_result}')
print('Confidence : ',round((1-(final_rmse - confidence_result[0])/confidence_result[0])*100,1),'%')

some_data = X_test[:4]
some_labels = y_test[:4]

results = optimized_final_model.predict(some_data)

print("Predictions:\t", results)
print("Labels:\t\t", list(some_labels))

Final RMSE :  47167.20449767728
95% Confidence Interval : [45168.60674493 49084.49169517]
Confidence :  95.6 %
Predictions:	 [494353.21       197922.67       214668.         164208.33333333]
Labels:		 [500001.0, 162500.0, 204600.0, 159700.0]
